# Rebar Strategy Backtest Report
Time Range: 2018-01-01 to 2023-12-31  
Underlying: RB Main Continuous  
Interval: 1 Min

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from strategy.rebar.main_strategy import RebarStrategy, MarketData, SignalType

# Mock Data Generator for Demonstration
def generate_mock_data(days=1000):
    dates = pd.date_range(start='2018-01-01', periods=days*240, freq='1min')
    price = 3500 + np.random.randn(len(dates)).cumsum()
    df = pd.DataFrame({
        'timestamp': dates,
        'open': price,
        'high': price + 5,
        'low': price - 5,
        'close': price + np.random.randn(len(dates)),
        'volume': np.random.randint(100, 10000, len(dates))
    })
    return df

print("Generating Mock Data...")
df = generate_mock_data(days=100) # Reduced for speed
print(f"Data Shape: {df.shape}")

In [ ]:
class BacktestEngine:
    def __init__(self, strategy, initial_capital=100000):
        self.strategy = strategy
        self.capital = initial_capital
        self.equity_curve = []
        
    def run(self, df):
        for i, row in df.iterrows():
            md = MarketData(
                symbol="RB",
                timestamp=row['timestamp'],
                open=row['open'],
                high=row['high'],
                low=row['low'],
                close=row['close'],
                volume=row['volume']
            )
            # Update Strategy
            self.strategy.on_tick(md)
            
            # Simulate random trade for demo
            # In real backtest, on_signal triggers trades and we track PnL here
            if i % 1000 == 0:
                self.capital *= (1 + np.random.uniform(-0.01, 0.015))
            
            self.equity_curve.append(self.capital)

strategy = RebarStrategy("strategy/rebar/params.json")
engine = BacktestEngine(strategy)
engine.run(df)

equity_df = pd.DataFrame({'equity': engine.equity_curve}, index=df['timestamp'])
equity_df.plot(figsize=(12, 6), title="Equity Curve")
plt.show()

In [ ]:
# Performance Metrics
total_return = (engine.equity_curve[-1] / engine.equity_curve[0]) - 1
print(f"Total Return: {total_return:.2%}")
# Add Sharpe, MaxDD logic here
